In [15]:
import os
import glob
import argparse
import pandas as pd
from multiprocessing import Pool

# split into 20 files
# in_file="/home/galaxy/project/m6AQTL/2019_10_10/transcriptome_snp/SNPiR/shell/GATK_best_practices/heart_2/heart_2_L7-raw_variants.txt"
# prefix = "heart_2_L7-"
# file_number = 20
####
parser = argparse.ArgumentParser()
parser.add_argument("--dataDir", "-d", help="The run directory")
parser.add_argument("--preifx", "-p", help="The prefix of input fastq file")
parser.add_argument("--thread", "-t", help="Thread number to run")
args = parser.parse_args()
#
data_dir = args.dataDir
prefix = args.preifx
in_file = "%s/%sraw_variants.txt" % (data_dir, prefix)
file_number = int(args.thread)
####
result_dir = os.path.join(os.path.dirname(in_file), "filter_mismatch_first6bp")
result_file = os.path.join(data_dir, "%srmhex.txt" % prefix)
##############################################################################################################
if not os.path.exists(result_dir):
    os.makedirs(result_dir)
os.chdir(result_dir)
total_lines = int(os.popen("wc -l %s" % in_file).read().split()[0])
line_number = int(total_lines / file_number) + 1
os.system("split -l %d %s %s" % (line_number, in_file, prefix))

# filter_mismatch_first6bp
def filter_mismatch_first6bp(in_file):
    os.chdir("/home/galaxy/project/m6AQTL/2019_10_10/transcriptome_snp/SNPiR/shell/SNPiR-master/")
    os.system("./filter_mismatch_first6bp.pl -infile %s -outfile %s/result_%s -bamfile %s/%srecal.bam" % (in_file, result_dir, os.path.basename(in_file), data_dir, prefix))
    
in_list = glob.glob("%s/%s*" % (result_dir, prefix))
pool = Pool(processes=file_number)
for x in in_list:
    pool.apply_async(filter_mismatch_first6bp, (x, ))
pool.close()
pool.join()    

# merge result
i_list = [x for x in glob.glob("%s/result_*" % result_dir) if "_tmp" not in x if "_failed" not in x]
os.system("cat %s | sort -k1,1 -k2,2n > %s" % (" ".join(i_list), result_file))